In [1]:
import pandas as pd

In [2]:
# Function to read stock data from an Excel file
def get_stock_data_from_excel(file_path):
    all_stock_data = pd.read_excel(file_path, sheet_name=None)
    stock_data = {}

    for sheet_name, data in all_stock_data.items():
        # Ensure 'Date' is treated as a datetime column and set as index
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        stock_data[sheet_name] = data

    return stock_data

In [3]:
def create_scrip_data(df, N):
    N_half = N // 2

    # Initialize the Scrip_Data DataFrame with required columns
    scrip_data = pd.DataFrame(index=df.index)
    scrip_data['Close'] = df['Close']
    scrip_data['Open'] = df['Open']
    scrip_data['High'] = df['High']
    scrip_data['Low'] = df['Low']
    scrip_data['Volume'] = df['Volume']

    # Calculate Previous N days Low
    scrip_data[f'Previous {N} days Low'] = df['Close'].rolling(window=N, min_periods=1).min().shift(1)

    # Calculate Today's Closing Higher for rows starting from N+1 onwards
    scrip_data['Today\'s Closing Lower'] = ''  # Initialize with NaN

    # Calculate Today's Closing Lower
    for i in range(N, len(scrip_data)):
        scrip_data.loc[scrip_data.index[i], 'Today\'s Closing Lower'] = (
            'YES' if scrip_data['Close'].iloc[i] < scrip_data[f'Previous {N} days Low'].iloc[i] else 'NO'
        )

    # Initialize the Enter Trade and Trade Number columns, but only calculate for rows starting from N+1 onwards
    scrip_data['Enter Trade'] = ''  # Initialize with NaN
    scrip_data['Trade Number'] = pd.NA  # Initialize with NaN

    # Calculate Trade taken in last N/2 days, Enter Trade, and Trade Number
    trade_taken = [0] * len(scrip_data)  # This will track the count of "ENTER" trades
    trade_number = 0

    for i in range(N, len(scrip_data)):
        last_n_half_days = scrip_data['Enter Trade'].iloc[i-N_half:i]
        trade_taken[i] = last_n_half_days[last_n_half_days == 'ENTER'].count()

        if scrip_data['Today\'s Closing Lower'].iloc[i] == 'YES' and trade_taken[i] == 0:
            scrip_data.loc[scrip_data.index[i], 'Enter Trade'] = 'ENTER'
            trade_number += 1
        else:
            scrip_data.loc[scrip_data.index[i], 'Enter Trade'] = 'NA'

        scrip_data.loc[scrip_data.index[i], 'Trade Number'] = trade_number

    # Add the new column: Trade taken in last N_half days, only for rows starting from N+1 onwards
    scrip_data[f'Trade taken in last {N_half} days'] = pd.NA  # Initialize with NaN
    for i in range(N, len(scrip_data)):
        scrip_data.loc[scrip_data.index[i], f'Trade taken in last {N_half} days'] = trade_taken[i]

    # Calculate N/2th Day, counting the entry day, only for rows starting from N+1 onwards
    scrip_data[f'{N_half}th Day'] = pd.NA  # Initialize with NaN
    for i in range(N, len(scrip_data)):
        if i + N_half - 1 < len(scrip_data):
            scrip_data.loc[scrip_data.index[i], f'{N_half}th Day'] = scrip_data.index[i + N_half - 1]

    return scrip_data

In [4]:
def create_trade_data(scrip_data, N):
    trade_data = []
    N_half = N // 2
    trades = scrip_data[scrip_data['Enter Trade'] == 'ENTER']

    for index, trade in trades.iterrows():
        trade_number = trade['Trade Number']
        entry_day = index
        nth_day = trade[f'{N_half}th Day']
        entry_price = trade['Close']

        # Adjust entry to be at the open of the next day
        next_day_index = scrip_data.index.get_loc(entry_day) + 1
        if next_day_index >= len(scrip_data):
            continue  # Skip if the next day is out of bounds

        next_day_data = scrip_data.iloc[next_day_index]
        entry_day = next_day_data.name  # Update entry_day to the next day
        entry_price = next_day_data['Open']

        # Calculate the N/2th day based on the adjusted entry day
        nth_day_index = next_day_index + (N_half - 1)
        if nth_day_index >= len(scrip_data):
            continue  # Skip if N_halfth day is out of bounds

        nth_day = scrip_data.index[nth_day_index]

        # Include the high for both entry and exit days in the calculation
        trade_period = scrip_data.loc[entry_day:nth_day]
        highest_point = max(trade_period['High'].max(), next_day_data['High'])
        lowest_point = min(trade_period['Low'].min(), next_day_data['Low'])

        highest_point_date = trade_period['High'].idxmax()
        lowest_point_date = trade_period['Low'].idxmin()

        long_potential = ((highest_point - entry_price) / entry_price) * 100
        short_potential = ((entry_price - lowest_point) / entry_price) * 100

        whipsaw_long = 'Yes' if long_potential < 0 else 'No'
        whipsaw_short = 'Yes' if short_potential < 0 else 'No'

        reached_first = 'Lowest' if lowest_point_date < highest_point_date else 'Highest'

        trade_data.append({
            'Trade Number': trade_number,
            'Entry Day': entry_day,  # Now reflects the next day
            f'{N_half}th Day': nth_day,  # Adjusted N_halfth day
            'Entry Price': entry_price,
            'Highest Point': highest_point,
            'Lowest Point': lowest_point,
            'Long Potential': long_potential,
            'Short Potential': short_potential,
            'Whipsaw Long': whipsaw_long,
            'Whipsaw Short': whipsaw_short,
            'Date of Highest Point': highest_point_date,
            'Date of Lowest Point': lowest_point_date,
            'Reached First?': reached_first
        })

    trade_df = pd.DataFrame(trade_data)

    return trade_df


In [5]:
def create_analysis(trade_data, N):
    N_half = N // 2

    # Calculate required metrics for the Analysis dataframe
    total_trades = len(trade_data)

    long_potential_30th_percentile = trade_data['Long Potential'].quantile(0.30)
    short_potential_30th_percentile = trade_data['Short Potential'].quantile(0.30)

    long_whipsaws_count = trade_data['Whipsaw Long'].value_counts().get('Yes', 0)
    short_whipsaws_count = trade_data['Whipsaw Short'].value_counts().get('Yes', 0)

    short_reached_first = trade_data['Reached First?'].value_counts().get('Highest', 0)
    long_reached_first = trade_data['Reached First?'].value_counts().get('Lowest', 0)

    # Create a dictionary to store the analysis data
    analysis_data = {
        'LookBack Period': N,
        'N/2': N_half,
        'Total Trades': total_trades,
        '30th Percentile Long Potential': long_potential_30th_percentile,
        '30th Percentile Short Potential': short_potential_30th_percentile,
        'Long Whipsaws Count': long_whipsaws_count,
        'Short Whipsaws Count': short_whipsaws_count,
        'Short Reached First': short_reached_first,
        'Long Reached First': long_reached_first
    }

    # Convert the dictionary to a DataFrame
    analysis_df = pd.DataFrame([analysis_data])

    return analysis_df

In [6]:
# Path to the Excel file containing stock data
file_path = '/content/drive/MyDrive/Data/DATA_FOR_TESTING.xlsx'
output_file_path = '/content/drive/MyDrive/Analysis/ENTRY AT THE OPEN/New Low Testing/AllStocks_Analysis.xlsx'


In [7]:
# Get stock data from Excel file
stock_data = get_stock_data_from_excel(file_path)

In [8]:
all_analysis_results = []

In [9]:
for scrip, df in stock_data.items():
    print(f"Processing {scrip}...")
    # Looping N from 5 to 55
    for N in range(5, 56):
        scrip_data = create_scrip_data(df, N=N)
        trade_data = create_trade_data(scrip_data, N=N)
        analysis_data = create_analysis(trade_data, N=N)

        # Add the Scrip name as the first column
        analysis_data.insert(0, 'Scrip', scrip)

        all_analysis_results.append(analysis_data)

Processing AARTIIND...
Processing ABB...
Processing ABBOTINDIA...
Processing ABCAPITAL...
Processing ABFRL...
Processing ACC...
Processing ADANIENT...
Processing ADANIPORTS...
Processing ALKEM...
Processing AMBUJACEM...
Processing APOLLOHOSP...
Processing APOLLOTYRE...
Processing ASHOKLEY...
Processing ASIANPAINT...
Processing ASTRAL...
Processing ATUL...
Processing AUBANK...
Processing AUROPHARMA...
Processing AXISBANK...
Processing BAJAJ-AUTO...
Processing BAJAJFINSV...
Processing BAJFINANCE...
Processing BALKRISIND...
Processing BALRAMCHIN...
Processing BANDHANBNK...
Processing BANKBARODA...
Processing BATAINDIA...
Processing BEL...
Processing BERGEPAINT...
Processing BHARATFORG...
Processing BHARTIARTL...
Processing BHEL...
Processing BIOCON...
Processing BOSCHLTD...
Processing BPCL...
Processing BRITANNIA...
Processing BSOFT...
Processing CANBK...
Processing CANFINHOME...
Processing CHAMBLFERT...
Processing CHOLAFIN...
Processing CIPLA...
Processing COALINDIA...
Processing COFORGE

In [10]:
# Combine all analysis results for all scrips into a single DataFrame
combined_analysis_df = pd.concat(all_analysis_results, ignore_index=True)

In [11]:
# Save the combined analysis results to a single Excel sheet
with pd.ExcelWriter(output_file_path) as writer:
    combined_analysis_df.to_excel(writer, sheet_name='Combined Analysis', index=False)

In [12]:
print(f"Processed data saved to {output_file_path}")

Processed data saved to /content/drive/MyDrive/Analysis/ENTRY AT THE OPEN/New Low Testing/AllStocks_Analysis.xlsx
